In [1]:
pip install requests python-dotenv

Note: you may need to restart the kernel to use updated packages.


In [2]:
import requests
import os
from dotenv import load_dotenv
from datetime import datetime, timedelta
import time
import json
import pandas as pd
import psycopg2

# loading environment variables (API key)

In [3]:
# load envrionment variables from the .env file
load_dotenv()

# get the API key from the .env file
api_key =os.getenv("API_KEY")

# getting latitude and longitude encodings for cities (list of cities in config file)

In [4]:
## function for getting lat & long encoding of cities

# Geocoding API endpoint
geocoding_url = "http://api.openweathermap.org/data/2.5/weather"

# List of cities
cities = ["Denver,CO,USA", "Austin,TX,USA", "Stuttgart,DE"]

# Loop through the cities and get their lat, lon
for city in cities:
    # Send GET request to the OpenWeatherMap Geocoding API
    response = requests.get(geocoding_url, params={
        'q': city,
        'appid': api_key
    })
    
    # Check if the request was successful
    if response.status_code == 200:
        data = response.json()
        lat = data['coord']['lat']
        lon = data['coord']['lon']
        print(f"City: {city} - Latitude: {lat}, Longitude: {lon}")
    else:
        print(f"Failed to get data for {city}")
        
##then store this information within the config file

City: Denver,CO,USA - Latitude: 39.7392, Longitude: -104.9847
City: Austin,TX,USA - Latitude: 30.2711, Longitude: -97.7437
City: Stuttgart,DE - Latitude: 48.7823, Longitude: 9.177


# API call function

In [5]:
# api call to get the current weather 
def get_weather(api_key, city, lat, lon, exclude='minutely,daily,hourly', units='imperial', lang='en'):
    # Build the base URL for the OneCall API
    url = f"https://api.openweathermap.org/data/3.0/onecall"
    
    # Prepare the parameters for the API call
    params = {
        'lat': lat,
        'lon': lon,
        'appid': api_key,
        'units': units,  # 'imperial' for Fahrenheit, 'metric' for Celsius
        'lang': lang      # Language for the response
    }
    
    # Add the 'exclude' parameter if it's provided
    if exclude:
        params['exclude'] = exclude
    
    # Make the API request
    response = requests.get(url, params=params)
    
    # Check if the request was successful
    if response.status_code == 200:
        data = response.json()
        # Print or process the data
        data['City']=city
        print(data)
        return data
    else:
        print(f"Error: {response.status_code}, {response.text}")
        return None

# Running the API call based upon cities in config file & then writing data to json file

In [6]:
# Step 1: Load the config file
with open('cities_config.json', 'r') as f:
    config_data = json.load(f)
    
weather_data = []

# Step 2: Loop through each city and use the data for API requests
for city in config_data['cities']:
    latitude = city['latitude']
    longitude = city['longitude']
    city_name = city['name']
    
    weather_data.append(get_weather(api_key, city_name, latitude, longitude))

{'lat': 39.7392, 'lon': -104.9847, 'timezone': 'America/Denver', 'timezone_offset': -25200, 'current': {'dt': 1739872951, 'sunrise': 1739886501, 'sunset': 1739925578, 'temp': 12.61, 'feels_like': 3.96, 'pressure': 1022, 'humidity': 83, 'dew_point': 8.89, 'uvi': 0, 'clouds': 100, 'visibility': 10000, 'wind_speed': 5.28, 'wind_deg': 29, 'wind_gust': 4.21, 'weather': [{'id': 804, 'main': 'Clouds', 'description': 'overcast clouds', 'icon': '04n'}]}, 'City': 'Denver,CO,USA'}
{'lat': 30.2711, 'lon': -97.7437, 'timezone': 'America/Chicago', 'timezone_offset': -21600, 'current': {'dt': 1739872951, 'sunrise': 1739884097, 'sunset': 1739924506, 'temp': 54.7, 'feels_like': 53.96, 'pressure': 1015, 'humidity': 87, 'dew_point': 50.92, 'uvi': 0, 'clouds': 100, 'visibility': 10000, 'wind_speed': 1.99, 'wind_deg': 104, 'wind_gust': 5.01, 'weather': [{'id': 804, 'main': 'Clouds', 'description': 'overcast clouds', 'icon': '04n'}]}, 'alerts': [{'sender_name': 'NWS Austin/San Antonio TX', 'event': 'Cold We

In [7]:
# File path within the "data" folder
file_path = 'data/raw_weather_data.json'

# Check if the file exists to decide whether to append or create new
if os.path.exists(file_path):
    # If the file exists, load the existing data, then append new data
    with open(file_path, 'r') as json_file:
        existing_data = json.load(json_file)
        existing_data.extend(weather_data)
    
    # Append to the file
    with open(file_path, 'w') as json_file:
        json.dump(existing_data, json_file, indent=4)
else:
    # If the file doesn't exist, create it and write the new data
    with open(file_path, 'w') as json_file:
        json.dump(weather_data, json_file, indent=4)

print(f"Data saved to {file_path}")

Data saved to data/raw_weather_data.json


# clean the weather data and write it to csv data file, all data and data to insert into database

In [8]:
# Function to convert Unix timestamp to local time
def convert_to_local_time(timestamp, offset):
    utc_time = datetime.utcfromtimestamp(timestamp)
    return utc_time + timedelta(seconds=offset)

In [9]:
# Initialize an empty list to store records
data = []

# Process each record
for record in weather_data:
    latitude, longitude = record["lat"], record["lon"]
    timezone = record["timezone"]
    timezone_offset = record["timezone_offset"]
    city = record['City']

    # Convert timestamps
    current_time = convert_to_local_time(record["current"]["dt"], timezone_offset)
    sunrise = convert_to_local_time(record["current"]["sunrise"], timezone_offset)
    sunset = convert_to_local_time(record["current"]["sunset"], timezone_offset)

    # Extract weather details
    temp = record["current"]["temp"]
    feels_like = record["current"]["feels_like"]
    pressure = record["current"]["pressure"]
    humidity = record["current"]["humidity"]
    dew_point = record["current"]["dew_point"]
    uvi = record["current"]["uvi"]
    clouds = record["current"]["clouds"]
    visibility = record["current"]["visibility"]
    wind_speed = record["current"]["wind_speed"]
    wind_deg = record["current"]["wind_deg"]
    wind_gust = record["current"].get("wind_gust", 0)
    weather = record["current"]["weather"][0]
    weather_id = weather["id"]
    weather_main = weather["main"]
    weather_description = weather["description"]

    # Handle alerts (if any)
    alerts = record.get("alerts", [])
    alert_messages = "; ".join([alert["event"] + ": " + alert["description"] for alert in alerts])

    # Add the record to the data list
    data.append({
        "latitude": latitude,
        "longitude": longitude,
        "timezone": timezone,
        "timezone_offset": timezone_offset,
        "city": city,
        "current_time": current_time,
        "sunrise": sunrise,
        "sunset": sunset,
        "temp_F": temp,
        "feels_like_F": feels_like,
        "humidity": humidity,
        "dew_point": dew_point,
        "uvi": uvi,
        "clouds": clouds,
        "visibility": visibility,
        "wind_speed_mph": wind_speed,
        "wind_deg": wind_deg,
        "wind_gust_mph": wind_gust,
        "weather_id": weather_id,
        "weather_main": weather_main,
        "weather_description": weather_description,
        "alerts": alert_messages
    })

# Create a DataFrame from the data list
df = pd.DataFrame(data)


In [48]:
df.head()

,latitude,longitude,timezone,timezone_offset,city,current_time,sunrise,sunset,temp_F,feels_like_F,...,uvi,clouds,visibility,wind_speed_mph,wind_deg,wind_gust_mph,weather_id,weather_main,weather_description,alerts
0,39.7392,-104.9847,America/Denver,-25200,"Denver,CO,USA",2025-02-18 03:02:31,2025-02-18 06:48:21,2025-02-18 17:39:38,12.61,3.96,...,0.00,100,10000,5.28,29,4.21,804,Clouds,overcast clouds,No alert
1,30.2711,-97.7437,America/Chicago,-21600,"Austin,TX,USA",2025-02-18 04:02:31,2025-02-18 07:08:17,2025-02-18 18:21:46,54.70,53.96,...,0.00,100,10000,1.99,104,5.01,804,Clouds,overcast clouds,Cold Weather Advisory: * WHAT...For the Cold W...
2,48.7823,9.1770,Europe/Berlin,3600,"Stuttgart,DE",2025-02-18 11:02:31,2025-02-18 07:26:11,2025-02-18 17:48:33,32.72,24.44,...,1.46,0,10000,10.36,70,0.00,800,Clear,clear sky,frost: There is a risk of frost (level 1 of 2)...


In [11]:
# File path within the "data" folder
file_path = 'data/clean_weather_data.csv'

# Check if the file exists to decide whether to append or create new
if os.path.exists(file_path):
    # If the file exists, load the existing data, then append new data
    existing_data = pd.read_csv(file_path)
    updated_data = pd.concat([existing_data, df], ignore_index=True)
    
    # Append to the file
    updated_data.to_csv(file_path, index=False)
else:
    # If the file doesn't exist, create it and write the new data
    df.to_csv(file_path, index=False)

print(f"Data saved to {file_path}")


Data saved to data/clean_weather_data.csv


In [12]:
# File path within the "data" folder
file_path = 'data/db_ready_data.csv'

# Check if the file exists to decide whether to append or create new
if os.path.exists(file_path):
    # If the file exists, load the existing data, then append new data
    existing_data = pd.read_csv(file_path)
    updated_data = pd.concat([existing_data, df], ignore_index=True)
    
    # Append to the file
    updated_data.to_csv(file_path, index=False)
else:
    # If the file doesn't exist, create it and write the new data
    df.to_csv(file_path, index=False)

print(f"Data saved to {file_path}")


Data saved to data/db_ready_data.csv


# write the cleaned data to a postgresql database

In [13]:
## need to open and read each row of the csv file 
file_path = 'data/db_ready_data.csv'

df_db = pd.read_csv(file_path)

In [14]:
## load environment variables and establish database connection

load_dotenv()

## get local database credentials
DB_NAME = os.getenv("DB_NAME")
DB_USER = os.getenv("DB_USER")
DB_PASSWORD = os.getenv("DB_PASSWORD")
DB_HOST = os.getenv("DB_HOST")
DB_PORT = os.getenv("DB_PORT")


def connect_db():
    try:
        conn = psycopg2.connect(
            dbname=DB_NAME,
            user=DB_USER,
            password=DB_PASSWORD,
            host=DB_HOST,
            port=DB_PORT)
        print("connected to postgresql on local host")
        return conn
    except psycopg2.Error as e:
        print("connection error: {e}")

In [15]:
# Insert or get Location_ID
def get_or_insert_location(cursor, lat, lon, city, timezone, tz_offset):
    cursor.execute(
        "SELECT Location_ID FROM Locations WHERE Lat=%s AND Long=%s;",
        (latitude, longitude)
    )
    location = cursor.fetchone()
    if location:
        print("location is already stored in the database")
        return location[0]
    else:
        cursor.execute(
            "INSERT INTO Locations (Lat, Long, City, Timezone, Timezone_offset) VALUES (%s, %s, %s, %s, %s) RETURNING Location_ID;",
            (lat, lon, city, timezone, tz_offset)
        )
        return cursor.fetchone()[0]

In [16]:
# Insert or get Weather_ID
def get_or_insert_weather(cursor, weather_id, main, description):
    cursor.execute(
        "SELECT Weather_ID FROM Weather where Weather_ID=%s;",
        (weather_id,)
    )
    weather = cursor.fetchone()
    if weather:
        print("weather is already stored in the database")
        return weather[0]
    else:
        cursor.execute(
            "INSERT INTO Weather (Weather_ID, Main, Description) VALUES (%s, %s, %s) RETURNING Weather_ID;",
            (weather_id, main, description)
        )
        return cursor.fetchone()[0]

In [17]:
# Insert Record

def insert_record(cursor, location_id, weather_id, row):
    cursor.execute(
    """
    INSERT INTO Records (Location_ID, Weather_ID, Local_time, Sunrise, Sunset, Temp_F, Feels_like_F, 
                        Humidity, Dew_Point, UVI, Clouds, Visibility, Wind_speed_mph, Wind_deg, Wind_gust_mph)
    VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s) RETURNING Record_ID;
    """,
    (location_id, weather_id, row['current_time'], row['sunrise'], row['sunset'], row['temp_F'],
        row['feels_like_F'], row['humidity'], row['dew_point'], row['uvi'], row['clouds'], row['visibility'], 
        row['wind_speed_mph'], row['wind_deg'], row['wind_gust_mph'])
    )
    print("record has been inserted into the table")
    return cursor.fetchone()[0]

In [49]:
# Insert Alert
def insert_alert(cursor, record_id, alert_description):
    cursor.execute(
        "INSERT INTO Alerts (Record_ID, Description) VALUES (%s, %s);",
        (record_id, alert_description)
    )
    print("alert has been inserted into the table")

In [50]:
conn = connect_db()

with conn.cursor() as cursor:
    for _, row in df_db.iterrows():
        location_id = get_or_insert_location(cursor, row['latitude'], row['longitude'], row['city'], row['timezone'], row['timezone_offset'])
        weather_id = get_or_insert_weather(cursor, row['weather_id'], row['weather_main'], row['weather_description'])
        record_id = insert_record(cursor, location_id, weather_id, row)
        
        if pd.notna(row['alerts']) and row['alerts'].strip():
            insert_alert(cursor, record_id, row['alerts'])
    conn.commit()
conn.close()
print("data inserted successfully & connection closed")

connected to postgresql on local host
location is already stored in the database
weather is already stored in the database
record has been inserted into the table
location is already stored in the database
weather is already stored in the database
record has been inserted into the table
alert has been inserted into the table
location is already stored in the database
weather is already stored in the database
record has been inserted into the table
alert has been inserted into the table
data inserted successfully


,latitude,longitude,timezone,timezone_offset,city,current_time,sunrise,sunset,temp_F,feels_like_F,...,uvi,clouds,visibility,wind_speed_mph,wind_deg,wind_gust_mph,weather_id,weather_main,weather_description,alerts
0,39.7392,-104.9847,America/Denver,-25200,"Denver,CO,USA",2025-02-18 03:02:31,2025-02-18 06:48:21,2025-02-18 17:39:38,12.61,3.96,...,0.00,100,10000,5.28,29,4.21,804,Clouds,overcast clouds,NaN
1,30.2711,-97.7437,America/Chicago,-21600,"Austin,TX,USA",2025-02-18 04:02:31,2025-02-18 07:08:17,2025-02-18 18:21:46,54.70,53.96,...,0.00,100,10000,1.99,104,5.01,804,Clouds,overcast clouds,Cold Weather Advisory: * WHAT...For the Cold W...
2,48.7823,9.1770,Europe/Berlin,3600,"Stuttgart,DE",2025-02-18 11:02:31,2025-02-18 07:26:11,2025-02-18 17:48:33,32.72,24.44,...,1.46,0,10000,10.36,70,0.00,800,Clear,clear sky,frost: There is a risk of frost (level 1 of 2)...


In [51]:
## delete the file
file_path = 'data/db_ready_data.csv'
if os.path.exists(file_path):
    os.remove(file_path)
    print("File deleted successfully")
else:
    print("File does not exist")

File deleted successfully
